#  Reading using kafka for Azure EventHub

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Eventhub namespace and required keys
#EH_NAMESPACE_NAME = "carao2023eventhub"
#EH_KEY_NAME = "readwrite"
#EH_KEY_VALUE = "eOlZaWO4qP/etM4GPkOcw98vbZXb6JLXPmUrpBJdSOQ="
# connection string of Event Hubs Namespace
connectionString = "Endpoint=sb://carao2023eventhub.servicebus.windows.net/;SharedAccessKeyName=readwrite;SharedAccessKey=eOlZaWO4qP/etM4GPkOcw98vbZXb6JLXPmUrpBJdSOQ=;EntityPath=vehicleposition"

ehConfig = {}
ehConfig['eventhubs.connectionString'] = connectionString
#You need to encryp connectionstrings
ehConfig['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)

EventHubdf = spark.readStream.format("eventhubs").options(**ehConfig).load()

In [0]:
display(EventHubdf.limit(10))

body,partition,offset,sequenceNumber,enqueuedTime,publisher,partitionKey,properties,systemProperties


In [0]:
print(EventHubdf.isStreaming)

print(EventHubdf.printSchema())

True
root
 |-- body: binary (nullable = true)
 |-- partition: string (nullable = true)
 |-- offset: string (nullable = true)
 |-- sequenceNumber: long (nullable = true)
 |-- enqueuedTime: timestamp (nullable = true)
 |-- publisher: string (nullable = true)
 |-- partitionKey: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- systemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

None


In [0]:
jsonSchema=StructType().add("id", StringType()).add("timestamp",TimestampType()).add("rpm", IntegerType()).add("speed", IntegerType()).add("kms", IntegerType())

In [0]:
#newEventHubDF= EventHubdf.selectExpr("CAST(body as STRING)")

#display(newEventHubDF.limit(20))

In [0]:
newEventHubDF=newEventHubDF.withColumn("vehicleJSON", from_json(col("body"), schema=jsonSchema))
newEventHubJSONDF= newEventHubDF.select("body", "vehicleJSON.*")
display(newEventHubJSONDF.limit(10))

body,id,timestamp,rpm,speed,kms


In [0]:
# Catch/Persist the data into a storage
newEventHubJSONDeltaDF = newEventHubJSONDF.selectExpr(
                  "id"      \
                  ,"timestamp"       \
                  ,"rpm"    \
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "dbfs:/vehicleposition/") \
.option("mergeSchema", "true") \
.start("dbfs:/Vehiclechkpoint_AzureEventHub_Delta")

#display(newEventHubJSONDeltaDF.limit(10))

In [0]:
dbutils.fs.ls("dbfs:/vehicleposition/")

Out[89]: [FileInfo(path='dbfs:/vehicleposition/__tmp_path_dir/', name='__tmp_path_dir/', size=0, modificationTime=1669359797000),
 FileInfo(path='dbfs:/vehicleposition/commits/', name='commits/', size=0, modificationTime=1669360958000),
 FileInfo(path='dbfs:/vehicleposition/metadata', name='metadata', size=45, modificationTime=1669359797000),
 FileInfo(path='dbfs:/vehicleposition/offsets/', name='offsets/', size=0, modificationTime=1669360956000),
 FileInfo(path='dbfs:/vehicleposition/sources/', name='sources/', size=0, modificationTime=1669359797000)]

In [0]:
%sql

-- Creating the table on delta location
CREATE TABLE IF NOT EXISTS Vehiclechkpoint_AzureEventHub_Delta
USING DELTA
LOCATION "dbfs:/Vehiclechkpoint_AzureEventHub_Delta"

In [0]:
%sql
--Check the number of records

SELECT COUNT(*) FROM Vehiclechkpoint_AzureEventHub_Delta;

SELECT COUNT(DISTINCT(id)) FROM Vehiclechkpoint_AzureEventHub_Delta;

--SELECT * FROM Vehiclechkpoint_AzureEventHub_Delta where id="80eea404-d0ef-41ba-a161-fac3e8030dbd" order by timestamp desc;

count(DISTINCT id)
40


In [0]:
#Trigger Every 3 seconds

newEventHubJSONDeltaDF = newEventHubJSONDF.selectExpr(
                  "id"      \
                  ,"timestamp"       \
                  ,"rpm"    \
                  ,"speed" \
                  ,"kms" ) \
.writeStream.format("delta") \
.outputMode("append") \
.option("checkpointLocation", "dbfs:/vehicleposition/") \
.option("mergeSchema", "true") \
.trigger( processingTime='3 seconds')\
.start("dbfs:/Vehiclechkpoint_AzureEventHub_Delta")

In [0]:
%sql

-- Creating the table on delta location
CREATE TABLE IF NOT EXISTS Vehiclechkpoint_AzureEventHub_Delta
USING DELTA
LOCATION "dbfs:/Vehiclechkpoint_AzureEventHub_Delta"